In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec  4 14:42:18 2020

@author: mark
"""

import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import datetime as dt

from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

from individual_analysis1 import create_small_df



In [ ]:
required=1

if required:
    small_df = create_small_df(size=10, cohort='dec_17')
    sdf = small_df['AmountPaid'].unstack(0).fillna(0).sort_index()
    monthly_sdf = sdf.groupby(pd.Grouper(freq='M')).sum()
    monthly_sdf.index = monthly_sdf.index.tz_localize(None)


pred_start_date = dt.datetime(2020, 7, 31,)
pred_end_date = dt.datetime(2020, 11, 30,)


for contractid in monthly_sdf.columns:
    one_contract = monthly_sdf[contractid]



### XGBoost

one_contract = one_contract.to_frame()
one_contract.index.rename('ds', inplace=True)
one_contract.rename(columns={one_contract.columns[0]:'y'}, inplace=True)





In [ ]:
def featurize(t):
    X = pd.DataFrame()

    X['day'] = t.index.day
    X['month'] = t.index.month
    X['quarter'] = t.index.quarter
    X['dayofweek'] = t.index.dayofweek
    X['dayofyear'] = t.index.dayofyear
    X['weekofyear'] = t.index.weekofyear
    y = t.y
    return X, y




In [ ]:
dataset = one_contract
featurize(dataset)[0].head()

X_train, y_train = featurize(
    dataset.loc[dataset.index < pd.to_datetime(pred_start_date)])
X_test, y_test = featurize(
    dataset.loc[dataset.index >= pd.to_datetime(pred_start_date)])

scaler = StandardScaler()
scaler.fit(X_train)
    
scaled_train = scaler.transform(X_train)
scaled_test = scaler.transform(X_test)

XGBOOST_model = XGBRegressor(n_estimators=7)